# MVD 4. cvičení

## 1. část - Načtení dat

Po rozbalení archive.zip uvidíte articles csv soubor. Tento soubor pochází z [Kaggle datasetů](https://www.kaggle.com/hsankesara/medium-articles) a obsahuje malé množství Medium článků k tématům ML, AI a data science. K úloze dnešního cvičení bude stačit využítí dat s názvy a obsahy článků (title a text).


### Příprava dat

Pro přípravu dat se použivá různá sekvence kroků. Je doporučeno na následující kroky vytvořit samostatnou funkci, aby bylo možné zpracovat i vyhledávaný výraz při testování. Dnešní cvičení by mělo obsahovat následující kroky:

1. Převést všechen text na lower case
2. Odstranění interpunkce a všech speciálních znaků (apostrof, ...)
3. Aplikace lemmatizátoru

Pozn.: Jedná se pouze o jednoduchý preprocessing, v praxi je často potřeba použití více kroků. Tato aplikace by měla například problém s čísly (desetinná čísla, čísla vyhledávaná slovně). 

Pro lemmatizaci použijte knihovnu spaCy.

In [ ]:
# Instalace spaCy z Jupyter Notebooku
import sys
!{sys.executable} -m pip install spacy

# Stažení modelu pro angličtinu
!{sys.executable} -m spacy download en

In [3]:
import spacy
lemmatizer = spacy.load('en_core_web_sm', disable=['parser', 'ner']) # NLTK
# Lemmatizace textu př.:  
# " ".join([token.lemma_ for token in lemmatizer(text)])

/home/pytel/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/pytel/.local/lib/python3.10/site-packages/spacy/language.py:1895: UserWarning: [W123] Argument disable with value ['parser', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


In [4]:
import csv
import json
import numpy as np

file_path = "data/articles.csv"
"""
with open(file_path, newline='') as f:
    reader = csv.reader(f)
    data_raw = list(reader)
"""

with open(file_path, 'r') as read_obj:
    dict_reader = csv.DictReader(read_obj)
    data_raw = list(dict_reader)

"""
for article in data_raw:
    print(json.dumps(article, indent=4))
"""

'\nfor article in data_raw:\n    print(json.dumps(article, indent=4))\n'

"author": "Justin Lee",
"claps": "8.3K",
"reading_time": "11",
"link": "https://medium.com/swlh/chatbots-were-the-next-big-thing-what-happened-5fc49dd6fa61?source=---------0----------------",
"title": "Chatbots were the next big thing: what happened? \u2013 The Startup \u2013 Medium",
"text":

In [5]:
def purge_chars(text, chars_to_remove):
    for c in list(chars_to_remove):
        text = text.replace(c, "")
    return text

def prep_text(text):
    # Převést všechen text na lower case
    text = text.lower()
    # Odstranění interpunkce a všech speciálních znaků (apostrof, ...)
    numbers = "1234567890"
    interpunction = ",.:;?!"
    chars = numbers + interpunction + '#^&@$€Łłþ→ø%+*/|\–—-\'’‘""”[]{}()'
    text = purge_chars(text, chars)
    # remove white spaces
    text = ' '.join(text.split())
    # Aplikace lemmatizátoru:
    return ' '.join([token.lemma_ for token in lemmatizer(text)])

def process_data(data):
    """
    data :
    [[Title, Text], ...] ->
    [[title, [word, ...]], ...]
    """
    ret = np.empty([len(data),2], dtype=object)
    for index, article in enumerate(data):
        ret[index, 0] = prep_text(article["title"])
        ret[index, 1] = prep_text(article["text"])
    return ret

In [ ]:
print(data_raw[0]["text"])
print("\t ->")
print(prep_text(data_raw[0]["text"]))

In [7]:
# process all data
data = process_data(data_raw)

In [ ]:
for article in data:
    print(article)
    break

In [8]:
print(data[:,0])

['chatbots be the next big thing what happen the startup medium'
 'python for datum science concept you may have forget'
 'automate feature engineering in python towards data science'
 'machine learn how to go from zero to hero freecodecamp'
 'reinforcement learning from scratch insight datum'
 'intuitively understand convolution for deep learning'
 'an intro to machine learning for designer ux collective'
 'the big list of dsml interview resource towards data science'
 'must know information theory concept in deep learning ai'
 'what I learn from interview at multiple ai company and startup'
 'from ballerina to ai researcher part I buzzrobot'
 'way to apply latent semantic analysis on largecorpus text on macos terminal jupyterlab and'
 'deep learning be go to teach we all the lesson of our life job be for machine'
 'machine learning be fun adam geitgey medium'
 'machine learning be fun part deep learning and convolutional neural network'
 'machine learning be fun part modern face reco

## 2. část - Vytvoření invertovaného indexu

Před další prací s textem je potřeba vytvořit invertovaný index, který poté usnadní práci. Invertovaný index bude slovník, kde klíčem bude slovo a hodnotou bude list s id dokumentů (index), které dané slovo obsahují.

Pozn.: Je potřeba vytvořit dva invertované indexy - jeden pro title a druhý pro text.

In [9]:
def flatten(l):
    return [item for sublist in l for item in sublist]

def split_to_word_lists(texts):
    return [text.split(' ') for text in texts]

def all_unique_words(word_lists):
    return np.unique(np.array(flatten(word_lists)))

def get_indexes(word_lists, word):
    indexes = []
    for index, words in enumerate(word_lists):
        if word in words:
            indexes.append(index) 
    return indexes

In [10]:
def make_reverse_indexing(texts):
    words_indexes = {}
    word_lists = split_to_word_lists(texts)
    for word in all_unique_words(word_lists):
        words_indexes[word] = get_indexes(word_lists, word)
    return words_indexes

In [11]:
def make_reverse_indexing(texts):
    words_indexes = {}
    word_lists = split_to_word_lists(texts)
    for index, words_list in enumerate(word_lists):
        #print(words_list)
        for word in words_list:
            indexes = words_indexes.get(word, set())
            indexes.add(index)
            words_indexes[word] = indexes
            #print("words_indexes[", word, "] ->", indexes)
    return words_indexes

In [12]:
def make_reverse_indexing(texts):
    words_indexes = {}
    word_lists = split_to_word_lists(texts)
    for index, words_list in enumerate(word_lists):
        #print(words_list)
        for word in words_list:
            indexes = words_indexes.get(word, [])
            indexes.append(index)
            words_indexes[word] = indexes
            #print("words_indexes[", word, "] ->", indexes)
    return words_indexes

In [13]:
[text.split(' ') for text in data[:,0]]

[['chatbots',
  'be',
  'the',
  'next',
  'big',
  'thing',
  'what',
  'happen',
  'the',
  'startup',
  'medium'],
 ['python',
  'for',
  'datum',
  'science',
  'concept',
  'you',
  'may',
  'have',
  'forget'],
 ['automate',
  'feature',
  'engineering',
  'in',
  'python',
  'towards',
  'data',
  'science'],
 ['machine',
  'learn',
  'how',
  'to',
  'go',
  'from',
  'zero',
  'to',
  'hero',
  'freecodecamp'],
 ['reinforcement', 'learning', 'from', 'scratch', 'insight', 'datum'],
 ['intuitively', 'understand', 'convolution', 'for', 'deep', 'learning'],
 ['an',
  'intro',
  'to',
  'machine',
  'learning',
  'for',
  'designer',
  'ux',
  'collective'],
 ['the',
  'big',
  'list',
  'of',
  'dsml',
  'interview',
  'resource',
  'towards',
  'data',
  'science'],
 ['must',
  'know',
  'information',
  'theory',
  'concept',
  'in',
  'deep',
  'learning',
  'ai'],
 ['what',
  'I',
  'learn',
  'from',
  'interview',
  'at',
  'multiple',
  'ai',
  'company',
  'and',
  'startu

In [14]:
titles = data[:,0]
texts = data[:,1]
title_indexing = make_reverse_indexing(titles)
text_indexing = make_reverse_indexing(texts)
print("Done")

Done


In [15]:
for word in title_indexing:
    print(word,":", title_indexing[word])

chatbots : [0, 86, 96, 96, 223, 273]
be : [0, 12, 12, 13, 14, 15, 18, 36, 56, 57, 58, 60, 61, 83, 86, 87, 98, 108, 113, 121, 123, 123, 128, 136, 138, 139, 140, 153, 169, 170, 216, 218, 224, 227, 231, 233, 237, 246, 258, 261, 264, 267, 273, 321, 330, 335]
the : [0, 0, 7, 12, 17, 19, 20, 21, 24, 25, 31, 32, 37, 43, 45, 46, 49, 51, 52, 61, 62, 66, 67, 75, 77, 79, 81, 82, 84, 86, 86, 91, 94, 96, 97, 99, 100, 102, 104, 105, 112, 117, 121, 124, 127, 128, 128, 130, 139, 140, 141, 142, 143, 147, 149, 154, 155, 163, 165, 175, 177, 180, 182, 182, 183, 189, 201, 204, 207, 208, 211, 217, 217, 218, 219, 224, 226, 229, 230, 230, 232, 241, 241, 241, 242, 244, 246, 248, 248, 248, 254, 255, 256, 258, 260, 262, 263, 263, 266, 267, 268, 273, 273, 274, 290, 298, 302, 303, 311, 330, 331, 334]
next : [0, 86, 241, 273, 334]
big : [0, 7, 32, 86, 91, 176, 202, 273, 292, 318]
thing : [0, 86, 102, 104, 109, 110, 247, 248, 273]
what : [0, 9, 41, 48, 86, 93, 129, 152, 160, 216, 240, 247, 267, 273]
happen : [0, 20,

## 3. část - Implementace TF-IDF

Připravení funkce pro výpočet TF-IDF po příchodu dotazu. Funkce *tf_idf* by měla pracovat s dotazem, jedním invertovaným indexem a s danými dokumenty. Vrátit by měla list obsahující skóre pro každý dokument.


$$ score(q,d) = TF\_IDF(q,d) = \sum\limits_{w \in q \cap d} c(w, q) c(w, d) log(\frac{M+1}{df(w)}) $$

$q$ ... dotaz<br>
$d$ ... dokument<br>
$c(w, q)$ ... kolikrát je slovo *w* v dotazu *q*<br>
$M$ ... celkový počet dokumentů<br>
$df(w)$ ... počet dokumentů, ve kterých se nachází slovo *w*

In [34]:
def sort_by_columm(array, columm=0, reverse_=False):
    return(sorted(array, key = lambda x: x[columm], reverse=reverse_)) 

def unique(array):
    return list(set(array))

def document_frequency(word, inverse_index):
    return len(unique(inverse_index.get(word, set())))

def c(word, inverse_index, document_index):
    return inverse_index.get(word, []).count(document_index)

def sim(query, inverse_index, documents, document_index):
    sume = 0
    M = len(documents)
    words = prep_text(query).split(' ')
    for word in unique(words):
        c1 = words.count(word)
        c2 = c(word, inverse_index, document_index)
        df = document_frequency(word, inverse_index)
        sume += c1 * c2 * np.log((M+1)/df)
    return sume

def tf_idf(query, inverse_index, documents):
    ret = []
    for index, document in enumerate(documents):
        title, text = document
        score = sim(query, inverse_index, documents, index)
        ret.append(score)
    return np.array(ret)

## 4. část - Použití a testování TF-IDF

Nyní lze získat skóre pro titulky nebo text. Následujícím krokem je sjednocení výsledného skóre pro ohodnocení celého dokumentu. V případě dvou hodnot si vystačíme s parametrem $\alpha$, který nám určuje jakou váhu má titulek a jakou samotný text dokumentu. <br>

$$ score(q,d) = \alpha \; TF\_IDF\_title(q,d) + (1-\alpha) \; TF\_IDF\_text(q,d) $$

Při nastavení parametru $\alpha$ na hodnotu 0.7 a vyhledávání dotazu "coursera vs udacity machine learning" by výsledky měly vypadat následovně:

![output](sample_output.png)

In [35]:
def score(query, documents, alpha=0.7):
    # return sorted keys of inverse index list by score
    titles = documents[:,0]
    texts = documents[:,1]
    title_indexing = make_reverse_indexing(titles)
    text_indexing = make_reverse_indexing(texts)
    tf_idf_title = tf_idf(query, title_indexing, documents)
    tf_idf_text = tf_idf(query, text_indexing, documents)
    return alpha*tf_idf_title + (1-alpha)*tf_idf_text

In [36]:
query = "coursera vs udacity machine learning"
scores = score(query, data)

5.823045895483019
1.6959115104379272
5.1298987149230735
5.823045895483019
1.3121863889661687
5.823045895483019
1.6959115104379272
5.1298987149230735
5.823045895483019
1.3121863889661687
5.823045895483019
1.6959115104379272
5.1298987149230735
5.823045895483019
1.3121863889661687
5.823045895483019
1.6959115104379272
5.1298987149230735
5.823045895483019
1.3121863889661687
5.823045895483019
1.6959115104379272
5.1298987149230735
5.823045895483019
1.3121863889661687
5.823045895483019
1.6959115104379272
5.1298987149230735
5.823045895483019
1.3121863889661687
5.823045895483019
1.6959115104379272
5.1298987149230735
5.823045895483019
1.3121863889661687
5.823045895483019
1.6959115104379272
5.1298987149230735
5.823045895483019
1.3121863889661687
5.823045895483019
1.6959115104379272
5.1298987149230735
5.823045895483019
1.3121863889661687
5.823045895483019
1.6959115104379272
5.1298987149230735
5.823045895483019
1.3121863889661687
5.823045895483019
1.6959115104379272
5.1298987149230735
5.823045895483

In [ ]:
print(scores)
print(np.argsort(scores))

In [ ]:
indexes = np.argsort(scores, )[::-1]
print(indexes)
scores_sorted = scores[indexes]
print(scores_sorted)

In [37]:
titles = np.array(data[:,0]).reshape(-1, 1)
texts = np.array(data[:,1]).reshape(-1, 1)
indexes = np.argsort(scores)[::-1]
scores_sorted = scores[indexes].reshape(-1, 1)
indexes_T = indexes.reshape(-1, 1)

sorted_data = np.concatenate((indexes_T, titles[indexes], texts[indexes], scores_sorted), axis=1)
print("Index: \t Title: \t Score:")
for article in sorted_data:
    print(article[0], "\t", article[1], "\t", article[3])

Index: 	 Title: 	 Score:
276 	 coursera vs udacity for machine learning hacker noon 	 31.64023124169509
67 	 every single machine learning course on the internet rank by your review 	 20.630531476579897
19 	 every single machine learning course on the internet rank by your review 	 20.630531476579897
143 	 every single machine learning course on the internet rank by your review 	 20.630531476579897
99 	 every single machine learning course on the internet rank by your review 	 20.630531476579897
327 	 in defense of skepticism about deep learning gary marcus medium 	 9.020900442507562
209 	 in defense of skepticism about deep learning gary marcus medium 	 9.020900442507562
13 	 machine learning be fun adam geitgey medium 	 7.2939142426100245
36 	 machine learning be fun adam geitgey medium 	 7.2939142426100245
47 	 machine learning in a week learn new stuff medium 	 7.274356135786702
117 	 the current state of machine intelligence shivon zilis medium 	 6.230807734115654
37 	 the current